## Project 3

import the libraries and set up the dataframe

In [78]:
import numpy as np 
import pandas as pd 
from itertools import combinations
file = r'C:\Users\chris\Git Hub Repositories\RandomCode\big_data_351\Groceries_dataset.csv'

df = pd.read_csv(file)
df.head()

,Member_number,Date,itemDescription
0,1808,21-07-2015,tropical fruit
1,2552,05-01-2015,whole milk
2,2300,19-09-2015,pip fruit
3,1187,12-12-2015,other vegetables
4,3037,01-02-2015,whole milk


In [138]:
grouped = df.groupby(['Member_number', 'Date'])['itemDescription'].apply(lambda x: list(set(x))).reset_index(name='Baskets')
grouped.head()


,Member_number,Date,Baskets
0,1000,15-03-2015,"[yogurt, whole milk, semi-finished bread, saus..."
1,1000,24-06-2014,"[pastry, whole milk, salty snack]"
2,1000,24-07-2015,"[misc. beverages, canned beer]"
3,1000,25-11-2015,"[hygiene articles, sausage]"
4,1000,27-05-2015,"[soda, pickled vegetables]"


In [226]:
SUPPORT_THRESHOLD = 0.0012
CONFIDENCE_THRESHOLD = 0.06 


In [ ]:
def find_frequent(data, support):
    counts = {}
    pass_1 = {}
    list_out = []
    for _, row in data.iterrows():
        for item in row['Baskets']:
            if item in counts:
                counts[item] += 1/len(data)
            else:
                counts[item] = 1/len(data)
    for item, count in counts.items():
        if count >= support:
            pass_1[item] = count
            list_out.append(item)
    return pass_1, list_out

In [223]:

def find_frequent_itemsets(itemset_candidates, data):
    valid_itemsets = {}
    for items in itemset_candidates:
        itemset = set(items)
        count = 0
        for transaction_items in data['Baskets']:
            if itemset.issubset(transaction_items):
                count += 1
        if count/len(grouped) >= SUPPORT_THRESHOLD:
            valid_itemsets[items] = count
   
    list_of_sets = [set(key) for key in valid_itemsets.keys()]    
    return list_of_sets


In [195]:
def create_sets(dict_items, strings, size):
    new_sets = set()  # Use a set to automatically handle duplicates
    for item_tuple in dict_items.keys():
        item_set = set(item_tuple)
        for string in strings:
            if string not in item_set:
                new_set = item_set.copy()
                new_set.add(string)
                if len(new_set) == size:
                    # Sort the set and convert to frozenset before adding
                    new_sets.add(frozenset(sorted(new_set)))

    # Convert each frozenset back to a regular set for the final list
    return [set(fs) for fs in new_sets]




In [227]:
def apriori(data, support):
    first = find_frequent(data, support)
    l1 = first[1]
    l1_dict = first[0]

    all_frequent_itemsets = {}
    all_frequent_itemsets.update(l1_dict)


    k = 2
    while True: 
        if k == 2:
            Ck = find_frequent_itemsets(list(combinations(l1,2)),data=grouped)
            Lk = {}
        else:             
            Ck = create_sets(Lk_minus_one, l1, size=k)  
            Lk = {}

        # Count the frequency of each candidate in Ck
        for transaction in data['Baskets']:  # Assume transactions is a list of all transactions
            for candidate in Ck:
                candidate_tuple = tuple(candidate)  # Convert set to tuple
                if candidate.issubset(set(transaction)):
                    if candidate_tuple in Lk:  # Use tuple as key
                        Lk[candidate_tuple] += 1
                    else:
                        Lk[candidate_tuple] = 1

        # Keep only those candidates in Lk that meet the minimum support threshold
        Lk = {itemset: count/len(data) for itemset, count in Lk.items() if count/len(data) >= support}



        if not Lk:  # If Lk is empty, no frequent itemsets of size k, break out of the loop
            break

        all_frequent_itemsets.update(Lk)

        # Prepare for the next iteration
        Lk_minus_one = Lk
        k += 1

    #df = pd.DataFrame(list(all_frequent_itemsets.items()), columns=['Baskets', 'Support'])

    

    return all_frequent_itemsets

ans = apriori(grouped, SUPPORT_THRESHOLD)
ans


        

{'yogurt': 0.08587850030074168,
 'whole milk': 0.15792287642853023,
 'semi-finished bread': 0.009490075519615037,
 'sausage': 0.06034886052262236,
 'pastry': 0.05172759473367631,
 'salty snack': 0.018779656485998768,
 'misc. beverages': 0.015772238187529215,
 'canned beer': 0.046915725456125025,
 'hygiene articles': 0.01370046113747241,
 'soda': 0.09710619528169467,
 'pickled vegetables': 0.008955423377664895,
 'frankfurter': 0.03775980752522883,
 'curd': 0.033683084942858994,
 'rolls/buns': 0.11000467820624187,
 'beef': 0.033950411013834066,
 'white bread': 0.02399251487001266,
 'whipped/sour cream': 0.04370781260442417,
 'other vegetables': 0.12210118291786384,
 'frozen vegetables': 0.02800240593463873,
 'butter': 0.035220209850965654,
 'sugar': 0.017710352202098482,
 'tropical fruit': 0.0677671589921806,
 'butter milk': 0.017576689166610947,
 'specialty chocolate': 0.01597273274076052,
 'detergent': 0.008621265788946056,
 'root vegetables': 0.06957160997126233,
 'dental care': 0.002

In [255]:
def generate_association_rules_df(itemsets, min_support, min_confidence):
    
    

    # Generate association rules and store them in a list of dictionaries
    rules_data = []
    for itemset in itemsets.keys():
        # Ensure the itemset is a set (needed for combinations)
        itemset = frozenset(itemset) if isinstance(itemset, tuple) else {itemset}

        for i in range(1, len(itemset)):
            for antecedent in combinations(itemset, i):
                antecedent = frozenset(antecedent)
                consequent = itemset - antecedent
                # print(antecedent, consequent)
                try:
                    antecedent_support = itemsets[str(set(antecedent))]
                except:
                    antecedent_support = itemsets
                try:
                    consequent_support = itemsets[str(set(consequent))]
                except:
                    consequent_support = itemsets
                if antecedent_support > 0 and consequent_support > 0:
                    confidence = itemsets[itemset] / antecedent_support
                    lift = confidence / consequent_support
                    if confidence >= min_confidence:
                        rule = {
                            'antecedent': antecedent,
                            'consequent': consequent,
                            'support_antecedent': antecedent_support,
                            'support_consequent': consequent_support,
                            'confidence': confidence,
                            'lift': lift
                        }
                        rules_data.append(rule)
    return rules_data



# Generate association rules and store in a DataFrame
generate_association_rules_df(ans, SUPPORT_THRESHOLD, CONFIDENCE_THRESHOLD)




TypeError: '>' not supported between instances of 'dict' and 'int'

# Support 

Definition: Support measures how frequently an item or an itemset appears in the dataset. It is calculated as the proportion of transactions in the dataset that contain the itemset. \
Use: High support means that the itemset is common in the dataset. In market basket analysis, a high support value indicates that the items are frequently bought together. \
Insights: The Apriori algorithm uses a minimum support threshold to eliminate itemsets with support lower than the threshold, as these are considered infrequent and thus not interesting for further analysis.


# Confidence

Definition: Confidence measures the likelihood of seeing the consequent in transactions under the condition that these transactions also contain the antecedent. It is calculated as the probability of the consequent given the antecedent. \
Use: Confidence provides insight into the reliability of the inference made by the rule. For example, in a rule like {Bread} => {Butter}, a high confidence value would mean that if Bread is bought, it is highly likely that Butter is also bought. \
Insights: The Apriori algorithm uses confidence to measure the strength of association between the antecedent and consequent of a rule. Rules with confidence values below a certain threshold are discarded.

# Lift 

Definition: Lift measures the strength of a rule over the random occurrence of the antecedent and consequent. It is the ratio of the observed support to that expected if the two sets were independent. \
Usee: A lift value greater than 1 indicates that the antecedent and consequent are positively correlated, while a value less than 1 indicates a negative correlation. A lift of exactly 1 implies independence. \
Insights: Lift helps in identifying the rules that have a real assocaition between the antecedent and consequent, rather than those that are simply a result of random chance.